In [41]:
import pandas as pd
import tensorflow as tf
from keras import layers, models
from sklearn.model_selection import train_test_split
from pathlib import Path
from tensorflow.keras import Sequential
from tensorflow.keras.layers import TextVectorization, Embedding, GlobalAveragePooling1D, Dense

In [37]:
# Step 1: Load and clean the dataset
data_path = Path("data/fake_job_postings.csv")
df = pd.read_csv(data_path)
df.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [14]:
# Drop rows with missing 'description' or target
df.dropna(subset=['description'], inplace=True)
df.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [24]:
# Create train/test split
# Use sklearn train_test_split with stratify on 'fraudulent'
# Combine title and description
df['text'] = df['title'].fillna('') + ' ' + df['description']

# Features and target
X = df['text'].values
y = df['fraudulent'].values

# Stratified train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [53]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.utils import class_weight
import numpy as np

# Assuming X_train and y_train are already prepared
# TextVectorization setup
vectorizer = layers.TextVectorization(
    max_tokens=10000,
    output_mode='int',
    output_sequence_length=300
)
vectorizer.adapt(X_train)

# Compute class weights to handle class imbalance
weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = {0: weights[0], 1: weights[1]}

# Create a text dataset
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32).prefetch(tf.data.AUTOTUNE)

# Build the model
model = models.Sequential([
    vectorizer,
    layers.Embedding(input_dim=10000, output_dim=128),
    layers.Bidirectional(layers.LSTM(64)),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall()
    ]
)

# Train
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=5,
    class_weight=class_weights
)


Epoch 1/5
447/447 ━━━━━━━━━━━━━━━━━━━━ 216s 450ms/step - accuracy: 0.8755 - loss: 0.5478 - precision_2: 0.1867 - recall_2: 0.5112 - val_accuracy: 0.9239 - val_loss: 0.2544 - val_precision_2: 0.3747 - val_recall_2: 0.8555
Epoch 2/5
447/447 ━━━━━━━━━━━━━━━━━━━━ 202s 451ms/step - accuracy: 0.9365 - loss: 0.1853 - precision_2: 0.4290 - recall_2: 0.9213 - val_accuracy: 0.9091 - val_loss: 0.2516 - val_precision_2: 0.3341 - val_recall_2: 0.8844
Epoch 3/5
447/447 ━━━━━━━━━━━━━━━━━━━━ 188s 421ms/step - accuracy: 0.9650 - loss: 0.0883 - precision_2: 0.5913 - recall_2: 0.9744 - val_accuracy: 0.9457 - val_loss: 0.1610 - val_precision_2: 0.4669 - val_recall_2: 0.8555
Epoch 4/5
447/447 ━━━━━━━━━━━━━━━━━━━━ 251s 530ms/step - accuracy: 0.9850 - loss: 0.0332 - precision_2: 0.7682 - recall_2: 0.9961 - val_accuracy: 0.9695 - val_loss: 0.0915 - val_precision_2: 0.6495 - val_recall_2: 0.8035
Epoch 5/5
447/447 ━━━━━━━━━━━━━━━━━━━━ 215s 426ms/step - accuracy: 0.9844 - loss: 0.0473 - precision_2: 0.7687 - rec